In [7]:
import sys
import os

project_root = '/Users/pradhnyesh/Documents/PJT-1'  # Your actual project root path
if project_root not in sys.path:
    sys.path.insert(0, project_root)


In [8]:
from src.data_processing import process_dataset_folder

base_folder = '/Users/pradhnyesh/Documents/PJT-1/dataset'
dataset_df = process_dataset_folder(base_folder, frames_save_root='/Users/pradhnyesh/Documents/PJT-1/saved_frames', out_csv_path='/Users/pradhnyesh/Documents/PJT-1/all_frame_features.csv')


Processing video: /Users/pradhnyesh/Documents/PJT-1/dataset/dry/MOV_12770.mp4 with label: dry
Processing video: /Users/pradhnyesh/Documents/PJT-1/dataset/dry/MOV_12764.mp4 with label: dry
Processing video: /Users/pradhnyesh/Documents/PJT-1/dataset/dry/MOV_12758.mp4 with label: dry
Processing video: /Users/pradhnyesh/Documents/PJT-1/dataset/dry/MOV_12759.mp4 with label: dry
Processing video: /Users/pradhnyesh/Documents/PJT-1/dataset/dry/MOV_12765.mp4 with label: dry
Processing video: /Users/pradhnyesh/Documents/PJT-1/dataset/dry/MOV_12771.mp4 with label: dry
Processing video: /Users/pradhnyesh/Documents/PJT-1/dataset/dry/MOV_12767.mp4 with label: dry
Processing video: /Users/pradhnyesh/Documents/PJT-1/dataset/dry/MOV_12773.mp4 with label: dry
Processing video: /Users/pradhnyesh/Documents/PJT-1/dataset/dry/MOV_12772.mp4 with label: dry
Processing video: /Users/pradhnyesh/Documents/PJT-1/dataset/dry/MOV_12766.mp4 with label: dry
Processing video: /Users/pradhnyesh/Documents/PJT-1/dataset/

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [11]:
# Train and evaluate Random Forest classifier
X = dataset_df[['mean_temp', 'std_temp', 'max_temp', 'min_temp']]
y = dataset_df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
print("\nTraining/testing data split:")
print(f"Train samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")

rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)
print("Random Forest model trained.")

y_pred = rf_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nTest Accuracy: {accuracy:.3f}")
print("Classification Report:\n", classification_report(y_test, y_pred))


Training/testing data split:
Train samples: 61, Test samples: 16
Random Forest model trained.

Test Accuracy: 0.812
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.62      0.77         8
           1       0.73      1.00      0.84         8

    accuracy                           0.81        16
   macro avg       0.86      0.81      0.81        16
weighted avg       0.86      0.81      0.81        16



In [13]:
import numpy as np
import cv2

In [ ]:
def extract_features_from_single_video(video_path, frame_skip=30):
    cap = cv2.VideoCapture(video_path)
    features = []
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_skip == 0:
            if len(frame.shape) == 3:
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            else:
                gray = frame
            temp_map = gray.astype(np.float32)
            mean_temp = np.mean(temp_map)
            std_temp = np.std(temp_map)
            max_temp = np.max(temp_map)
            min_temp = np.min(temp_map)
            features.append([mean_temp, std_temp, max_temp, min_temp])
        frame_count += 1
    cap.release()
    features = np.array(features)
    avg_features = features.mean(axis=0)
    return avg_features

def predict_soil_moisture(video_path, model, frame_skip=30):
    feature_vector = extract_features_from_single_video(video_path, frame_skip)
    feature_vector = feature_vector.reshape(1, -1)  # Reshape for sklearn input
    predicted_label = model.predict(feature_vector)[0]
    label_map = {0: 'dry', 1: 'wet'}
    return label_map.get(predicted_label, "Unknown")

test_video_path = "/Users/pradhnyesh/Documents/PJT-1/dataset/dry/MOV_12747.mp4"
prediction = predict_soil_moisture(test_video_path, rf_clf, frame_skip=30)
print(f"Predicted soil moisture for video '{os.path.basename(test_video_path)}': {prediction}")


Predicted soil moisture for video 'MOV_12747.mp4': dry


/opt/miniconda3/envs/pjt1/lib/python3.13/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
